In [1]:
import numpy as np
import pandas as pd
import pickle
import os
import re
import nltk
import string
from pymorphy2 import MorphAnalyzer
import seaborn as sns
from collections import namedtuple

sns.set_style('whitegrid')
sns.despine()

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
features = namedtuple('features', ['word', 'part_of_speech', 'tonality', 'source', 'extra'])
with open('Desktop/ML/NLPSchool/aot/dict_2py.pickle', 'rb') as fin:
    Dict = pickle.load(fin)

In [3]:
filenames = os.listdir('Desktop/ML/NLPSchool/train_data/')
opin_ptn = re.compile('[a-z0-9]*\.opin\.txt')
enti_ptn = re.compile('[a-z0-9]*\.ann')
opinion_names = filter(lambda s: re.match(opin_ptn, s) is not None, filenames)
entities_names = filter(lambda s: re.match(enti_ptn, s) is not None, filenames)
articles_names = filter(lambda s: re.match(enti_ptn, s) is None and re.match(opin_ptn, s) is None, filenames)

In [4]:
articles = []

for name in articles_names:
    with open('Desktop/ML/NLPSchool/train_data/' + name, 'r') as fin:
        articles.append(fin.read())

In [5]:
print articles[0][:100]

{Author, Unknown} 

{Author, Unknown} СМИ Ирана: Финляндия не хочет вст�


In [6]:
sentences = []
for article in articles:
    splitted = nltk.sent_tokenize(article.decode('utf-8'))
    
    sentences.extend(splitted)
    
for s in sentences[:5]:
    print s

{Author, Unknown} 

{Author, Unknown} СМИ Ирана: Финляндия не хочет вступать в НАТО из страха перед Россией?
{Author, Unknown} inosmi.ru

{Author, Unknown} 5-го, 2016


{Author, Unknown} Зачем американцам понадобилось перебросить 250 своих военных на северо-восток Сирии, к окрестностям города Румийлан, разбирался обозреватель Khorasan (
{Author, Unknown} 30.04) Алиреза Резахах (Alireza Rezakhah).
{Author, Unknown} Согласно заявлениям из Пентагона, цель США — способствовать освобождению от боевиков ИГИЛ стратегически важного города Ракка.
{Author, Unknown} Однако в Дамаске действия Вашингтона резко осудили и назвали покушением на территориальный суверенитет Сирии.
{Author, Unknown} Примерно в аналогичном ключе высказалась и Москва, где «несогласованные с Дамаском» шаги Белого дома назвали фактором, вызывающим озабоченность.


In [7]:
words = []
morph = MorphAnalyzer()
for s in sentences:
    sentence_words = nltk.word_tokenize(s)
    sentence_words = [w for w in sentence_words if w not in string.punctuation]
    sentence_words = [w.lower() for w in sentence_words if w not in ['Author', 'Unknown']]
    sentence_words = [morph.parse(w)[0].normal_form for w in sentence_words if morph.parse(w)[0] is not None]
    words.append(sentence_words)

In [8]:
print words[0][0], words[0][1], words[0][2]

сми иран финляндия


In [9]:
emph = []
for sent in words:
    tmp = []
    for word in sent:
        if word in Dict.keys():
            if len(Dict[word]) > 1:
                tonality = 0
                for i in range(len(Dict[word])):
                    w_tonality = Dict[word][i].tonality
                    if w_tonality == 'positive':
                        tonality += 1.
                    elif w_tonality == 'negative':
                        tonality -= 1.
                tonality /= len(Dict[word])
                tmp.append(tonality)
            else:
                w_tonality = Dict[word][0].tonality
            if w_tonality == 'positive':
                tmp.append(1.)
            elif w_tonality == 'negative':
                tmp.append(-1.)
            else:
                tmp.append(0.)
    if len(tmp) > 0:
        emph.append(sum(tmp) / float(len(tmp)))
    else:
        emph.append(0)
    
print emph[:10]

[-1.0, 0, 0.66666666666666663, -1.0, -1.0, -0.3125, -0.83333333333333337, 0.5, 1.0, 0]


In [10]:
for s, mark in zip(sentences, emph):
    print mark, s

-1.0 {Author, Unknown} 

{Author, Unknown} СМИ Ирана: Финляндия не хочет вступать в НАТО из страха перед Россией?
0 {Author, Unknown} inosmi.ru

{Author, Unknown} 5-го, 2016


{Author, Unknown} Зачем американцам понадобилось перебросить 250 своих военных на северо-восток Сирии, к окрестностям города Румийлан, разбирался обозреватель Khorasan (
{Author, Unknown} 30.04) Алиреза Резахах (Alireza Rezakhah).
0.666666666667 {Author, Unknown} Согласно заявлениям из Пентагона, цель США — способствовать освобождению от боевиков ИГИЛ стратегически важного города Ракка.
-1.0 {Author, Unknown} Однако в Дамаске действия Вашингтона резко осудили и назвали покушением на территориальный суверенитет Сирии.
-1.0 {Author, Unknown} Примерно в аналогичном ключе высказалась и Москва, где «несогласованные с Дамаском» шаги Белого дома назвали фактором, вызывающим озабоченность.
-0.3125 {Author, Unknown} Почему же и в Москве и Дамаске, которые также ведут борьбу против ИГ в Сирии, действия Вашингтона вызвали с